In [2]:
from ReadWriteGDocs import write_gdoc, read_gdoc_range, write_gdoc_cell
# write_gdoc(data, ws_name, sht_name, row, col)
# read_gdoc_range(ws_name,sht_name="Sheet1",startrow=1,startcol=1, endrow=1, endcol=1)

import pandas as pd
import numpy as np

input_ws = ["NY Mini 2018 Score Input - Men's (Day 2) (Responses)", 
            "NY Mini 2018 Score Input - Women's (Day 2) (Responses)"]
output_ws = ["GOLD NY Mini 2017 - Day 2 Pools",
             "SILVER NY Mini 2017 - Day 2 Pools", 
             "BRONZE NY Mini 2017 - Day 2 Pools"]

c=read_gdoc_range(input_ws[1], "Form Responses 1", 1,2,1,12)
c=c[0]

m_pools = ['01', '02', '03', '04', '07', '08', '12', '13', '14']
f_pools = ['05', '06', '09', '10', '11', '15', '16', '17', '18']
# 0-3 are gold pools, 4-6 are silver, 7-8 are bronze 

# start processing from row 2
# keep track of rows processed using processed

processed = 0

Read data from Form Responses

Write to corresponding GDocs

First, figure out what order to write the scores.
Then, write the scores.

In [7]:
m_in = pd.DataFrame(read_gdoc_range(input_ws[0], "Form Responses 1", processed + 2,2, 100, 12), 
                    columns=c)
m_in.replace('', np.nan, inplace=True)
m_in.dropna(inplace=True, how='all')

from math import isnan

print ("Need to process " + str(len(m_in)) + " games")

for index,row in m_in.iterrows():
    sheet = str(row['Court Number']) + " (M)"
    game = int(row['Game Number'])
    
    #determine gold, silver, bronze
    pool_index = m_pools.index(row['CourtNumber'][2:4])
    bracket_num = -1;
    
    if (pool_index <= 3):
        bracket_num = 0;
    elif (pool_index <= 6):
        bracket_num = 1;    
    elif (pool_index < len(m_pools)):
        bracket_num = 2;
    else:
        print ("Error for " + sheet + " Game " + str(game))
        break;
    
    print("Processing " + sheet + " Game " + str(game))
    
    T1 = row['Team 1 Name']
    
    scores = np.array([row['Team 1 Score'], 0, 0, row['Team 2 Score']])
    scores = np.append(scores,[0, row['Team 1 Score 2'], 0, 0, row['Team 2 Score 2']])
        
    #read game team name (first one only)
    
    team_names = read_gdoc_range(output_ws[bracket_num], sheet, 13+game, 3, 13+game, 3)
    
    if T1 != team_names[0][0]:
        scores = scores[::-1]
            
    for i in range(len(scores)):
        write_gdoc_cell(scores[i], output_ws[bracket_num], sheet, 13+game, 17+i)
        
print("Number of records processed (men's): " + str(len(m_in)))

processed = processed + len(m_in)

print("Total Processed: " + str(processed))

Number of records processed (men's): 0
Total Processed: 0
